In [1]:
import numpy as np

class Mapa:
    def __init__(self,nodes,neighbors,begin,end):
        self.nodes=nodes
        self.begin=begin
        self.end=end
        self.neighbors=neighbors
        
    def setStart(begin):
        self.begin=begin
            
    def setEnd(end):
        self.end=end

In [13]:
def euklid(a,b):
    sums=0
    for i in range(len(a)):
        sums
        sums+=(b[i]-a[i])**2
    return sums**(1/2)

def manhatan(a,b):
    sums=0
    for i in range(len(a)):
        sums+=abs(a[i]-b[i])
    return sums
        
def maximum(a,b):
    list1=[]
    for i in range(len(a)):
        list1.append(abs(a[i]-b[i]))
    return max(list1)

def lenght_of_road(list1,points,metric):
    sums=0
    for i in range(len(list1)-1):
        sums+=metric(points[list1[i]],points[list1[i+1]])
    return sums

In [23]:
def solve_aco(maps,metric=euklid,heuristic_coefficient=1,feromones_coefficient=1):
    ants=5
    repetitions=10
    #na podstawie mapy tworzymy słownik wszystkich możliwych ścieżek przypisując im bazową liczbę fermonów: 1
    every_trial_in_graph={}
    for i in maps.nodes:
        node1=i[0]
        for j in maps.neighbors[i[0]]:
            node2=j
            single_trial=frozenset({node1,node2})
            if single_trial not in every_trial_in_graph.keys():
                every_trial_in_graph.update({single_trial:1})

    #algorytm główny            
    for repetition in range(repetitions):
        #lista która trzyma  trasy wszystkich mrówek w danym powtórzeniu
        list_of_tracks_of_every_ant=[]
        last_repetiton_roads={}
        for ant in range(ants):
            current_node=maps.begin
            nodes_visited_by_ant=[maps.begin]
            while current_node!=maps.end:
                #pobieramy listę sąsiadów wierzchołka w którym jesteśmy i tworzymy liste wierzchołków z których będziemy wybierać
                neighbour_nodes=list(maps.neighbors[current_node])
                nodes_to_choose=[]
                #dodajemy wierzchołki w których mrówka nie była
                for node in neighbour_nodes:
                    if node not in nodes_visited_by_ant:
                        nodes_to_choose.append(node)
                #jeśli widzimy ostatni punkt to go od razu wybieramy
                if maps.end in nodes_to_choose:
                    choosen_node=maps.end
                
                else:
                    #jeśli wszystkie wierzchołki odwiedzone, to wybieramy z wszystkich dostępnych
                    if nodes_to_choose==[]:
                        nodes_to_choose=neighbour_nodes
                    sum_for_probabilty=0
                    #liczymy sume wszystkich wskaźników
                    for node in nodes_to_choose:
                        heuristic=metric(maps.nodes[node],maps.nodes[current_node])/metric(maps.nodes[node],maps.nodes[maps.end])
                        feromones=every_trial_in_graph[frozenset({current_node,node})]
                        sum_for_probabilty+=(heuristic**heuristic_coefficient)*(feromones**feromones_coefficient)
                    random_for_probaiblity=np.random.random_sample()
                    chance_sum=0
                    #wybieramy punkt do którego pójdzie mrówka
                    for node in nodes_to_choose:
                        heuristic=metric(maps.nodes[node],maps.nodes[current_node])/metric(maps.nodes[node],maps.nodes[maps.end])
                        feromones=every_trial_in_graph[frozenset({current_node,node})]
                        chance_sum+=(heuristic**heuristic_coefficient)*(feromones**feromones_coefficient)/sum_for_probabilty
                        if chance_sum>random_for_probaiblity:
                            choosen_node=node
                            break    
                nodes_visited_by_ant.append(choosen_node)
                current_node=choosen_node
            #tworzymy listę wszystkich ścieżek które przeszły mrówki - elementami są ścieżki 
            road_of_current_ant_using_trials=[]
            for k in range(len(nodes_visited_by_ant)-1):
                road_of_current_ant_using_trials.append(frozenset({nodes_visited_by_ant[k],nodes_visited_by_ant[k+1]}))
            list_of_tracks_of_every_ant.append(road_of_current_ant_using_trials)
            #jeśli to ostatnia próba to zliczamy ile razy dana trasa została pokonana
            if repetition==(repetitions-1):
                if tuple(nodes_visited_by_ant) in last_repetiton_roads:
                    last_repetiton_roads[tuple(nodes_visited_by_ant)]+=1
                else:
                    last_repetiton_roads[tuple(nodes_visited_by_ant)]=1
        
        #aktualizacja feromonów
        #najpierw połowa wyparowuje
        for trial in every_trial_in_graph:
            every_trial_in_graph[trial]=every_trial_in_graph[trial]*0.5
        #teraz dodajemy tam gdzie mrówki chodziły
        for track in list_of_tracks_of_every_ant:
            track_length=0
            for trial_nodes in track:
                trial_as_list=list(trial_nodes)
                track_length+=metric(maps.nodes[trial_as_list[0]],maps.nodes[trial_as_list[1]])
            for trial_nodes in track:
                every_trial_in_graph[trial_nodes]+=5/(track_length-metric(maps.nodes[maps.begin],maps.nodes[maps.end]))**3
    
    print(last_repetiton_roads)

In [4]:
lista={
    "A":(6,0),
    "B":(2,2),
    "C":(8,2),
    "D":(4,4),
    "E":(6,6),
    "F":(4,8),
    "G":(6,10),
    "F":(4,8),
    "H":(8,8),
    "I":(2,6)
}

sasiedzi={
    "A":("B","C","D"),
    "B":("A","I"),
    "C":("A","E"),
    "D":("A","I","E"),
    "E":("F","G","H","C","D",),
    "F":("I","E"),
    "G":("E","H"),
    "H":("E","G"),
    "I":("F","D"),
}

In [8]:
lista2={
    "A":(4,1), 
    "B":(2,4), 
    "C":(8,1), 
    "D":(6,3), 
    "E":(2,7), 
    "F":(4,6), 
    "G":(10,4), 
    "F":(4,6), 
    "H":(12,8), 
    "I":(8,9), 
    "J":(7,6), 
    "K":(5,10), 
    "L":(11,11), 
    "M":(13,3), 
    "N":(12,1) 
    } 

sasiedzi2={
    "A":("B","C","D","E"), 
    "B":("A","aK"), 
    "C":("A","G","L"), 
    "D":("A","F","G"), 
    "E":("A","F","J"), 
    "F":("D","E","J"), 
    "G":("C","D","K","M"), 
    "H":("J","L"), 
    "I":("J","L","N"), 
    "J":("E","F","H","I"), 
    "K":("B","G"), 
    "L":("C","H","I"), 
    "M":("G","N"), 
    "N":("I","M"), 
    }   

In [5]:
plansza1= Mapa(lista,sasiedzi,'A',"G")

In [9]:
plansza2= Mapa(lista2,sasiedzi2,'A',"H")

In [24]:
for i in range(18):
    solve_aco(plansza2,metric=euklid,heuristic_coefficient=0.1,feromones_coefficient=1+(i+1)*0.1)

{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'E', 'J', 'H'): 4, ('A', 'B', 'K', 'G', 'D', 'F', 'J', 'H'): 1}
{('A', 'D', 'F', 'J', 'H'): 4, ('A', 'D', 'F', 'E', 'J', 'H'): 1}
{('A', 'E', 'J', 'H'): 5}
{('A', 'E', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'E', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 4, ('A', 'E', 'J', 'H'): 1}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'D', 'F', 'J', 'H'): 5}
{('A', 'E', 'J', 'H'): 5}


In [26]:
for i in range(18):
    solve_aco(plansza1,metric=euklid,heuristic_coefficient=0.1,feromones_coefficient=1+(i+1)*0.1)

{('A', 'C', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'D', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'D', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'D', 'E', 'G'): 5}
{('A', 'D', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'D', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'C', 'E', 'G'): 5}
{('A', 'D', 'E', 'G'): 5}


[]=False

In [42]:
lenght_of_road(['A','H'],lista2,euklid)

10.63014581273465

In [74]:
lenght_of_roadn(['A','D','F','J','H'],lista2,euklid)

14.819143207344684

In [73]:
lenght_of_road(['A','E','J','H'],lista2,euklid)

16.808739641064047

In [72]:
lenght_of_road(['A','C','L','H'],lista2,euklid)

17.60258416907893